### Trajectory Clustering

In [1]:
#import math
import torch
import gpytorch
import numpy as np
#from sklearn.model_selection import train_test_split
from kneed import KneeLocator
from sklearn.cluster import KMeans
#from sklearn.metrics import silhouette_score
from gpytorch.kernels import RQKernel as RQ, RBFKernel as SE, \
PeriodicKernel as PER, ScaleKernel, LinearKernel as LIN, MaternKernel as MAT, \
SpectralMixtureKernel as SMK, PiecewisePolynomialKernel as PPK, CylindricalKernel as CYL
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib
from matplotlib import pyplot as plt
import pandas as pd
import random
import mobileDataToolkit.preprocessing_v2 as preprocessing
import mobileDataToolkit.analysis as analysis
import mobileDataToolkit.methods as methods
import mobileDataToolkit.metrics as metrics
import utils.GP as GP
import utils.helper_func as helper_func
import geopandas as gpd
import skmob
import skmob.preprocessing.detection
import skmob.preprocessing.clustering
import warnings
warnings.filterwarnings("ignore")

In [3]:
in_path = "C:/Users/stlp/OneDrive - UW/GPR/Data/newAllTrips_withmetrics.csv"
c_path = "C:/Users/stlp/OneDrive - UW/GPR/Data/newCompressedTrips.csv"
all_path = "C:/Users/stlp/OneDrive - UW/GPR/Data/10_users_all_obs_raw.csv"

# Mobility metrics dataset preprocessing
m_df = pd.read_csv(c_path, header=0)
m_df = m_df.dropna()

# Filter out trips with unrealistic speeds, durations, and number of points
m_df = m_df[(m_df['vel_avg'] < 60) & #no faster than 60 m/s (as the crow flies)
            (m_df['time_total'] < 7200) & # no longer than 2 hours
            (m_df['npoints'] > 4) & # at least 5 points for modeling
            (m_df['StartDay'] == m_df['EndDay']) # start day and end day must be the same
            ]

m_df = m_df[m_df['Id_perc'] != 2141084034]

feats = m_df[['vel_avg', 'distanceTotal', 'time_total', 'hcr', 'vcr', 'npoints', 'sr']]

In [ ]:
import similaritymeasures

n_points = 100

# Get regularly sampled points along the trip from linestring object
def get_points_from_linestring(ls, n_points=n_points):
    # Get the length of the linestring
    length = ls.length
    # Get the distance between each point
    dist = length / n_points
    # Get the points
    points = [ls.interpolate(i * dist) for i in range(n_points)]
    # Convert points to a geodataframe
    points_gdf = gpd.GeoDataFrame(geometry=points)
    return points_gdf

# Do a for loop to get the points for each trip
for i in range(len(similar_trips_ls)):
    similar_trips_points.append(get_points_from_linestring(similar_trips_ls[i]))
    

# Get the points along the trip
trip1_points = get_points_from_linestring(trip1_ls)
similar_trips_points = [get_points_from_linestring(i) for i in similar_trips_ls]

trip1_points_arr = np.zeros((100, 2))
trip1_points_arr[:, 0] = trip1_points.geometry.x
trip1_points_arr[:, 1] = trip1_points.geometry.y

similar_trips_points_arr = np.zeros((100, 2))
similar_trips_points_arr[:, 0] = similar_trips_points[0].geometry.x
similar_trips_points_arr[:, 1] = similar_trips_points[0].geometry.y

#trip1_pts = np.zeros((n_points = 100, 2))
#num_data[:, 0] = trip1_points.geometry.x
#num_data[:, 1] = trip1_points.geometry.y

# quantify the difference between the two curves using PCM
pcm = similaritymeasures.pcm(trip1_points_arr, similar_trips_points_arr)

# quantify the difference between the two curves using
# Discrete Frechet distance
df = similaritymeasures.frechet_dist(trip1_points_arr, similar_trips_points_arr)

# quantify the difference between the two curves using
# area between two curves
area = similaritymeasures.area_between_two_curves(trip1_points_arr, similar_trips_points_arr)

# quantify the difference between the two curves using
# Curve Length based similarity measure
cl = similaritymeasures.curve_length_measure(trip1_points_arr, similar_trips_points_arr)

# quantify the difference between the two curves using
# Dynamic Time Warping distance
dtw, d = similaritymeasures.dtw(trip1_points_arr, similar_trips_points_arr)

# mean absolute error
mae = similaritymeasures.mae(trip1_points_arr, similar_trips_points_arr)

# mean squared error
mse = similaritymeasures.mse(trip1_points_arr, similar_trips_points_arr)

# print the results
print(pcm, df, area, cl, dtw, mae, mse)

In [ ]:
import shapely
from shapely.geometry import LineString

# Create linestring objects for each trip
trip1_ls = LineString(zip(tr_df.data[tr_df.data['trip_ID'] == trip1['trip_ID'].iloc[0]]['lng'], tr_df.data[tr_df.data['trip_ID'] == trip1['trip_ID'].iloc[0]]['lat']))
similar_trips_ls = [LineString(zip(tr_df.data[tr_df.data['trip_ID'] == j]['lng'], tr_df.data[tr_df.data['trip_ID'] == j]['lat'])) for j in np.setdiff1d(tr_df.data['trip_ID'].unique(), trip1['trip_ID'].unique())]

# Calculate the distance between trip1 and each similar trip
distances = [trip1_ls.distance(i) for i in similar_trips_ls]
print(distances)